In [ ]:
import os
import cv2
import numpy as np
import nibabel as nib
import time

scale_factor = 4
input_dir = r""
output_dir = r""

for file_name in os.listdir(input_dir):
    if file_name.endswith(".nii.gz"):
        file_path = os.path.join(input_dir, file_name)
        print("Processing:", file_path)
        input_data = nib.load(file_path)
        input_img = input_data.get_fdata()
        layer_thickness = input_data.header.get_zooms()[2]
        spacing = input_data.header.get_zooms()[:2]
        
        upsampled_layers = []
        for i in range(input_img.shape[2]):
            input_layer = input_img[:, :, i]
            input_layer = cv2.normalize(input_layer, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
            new_shape = (input_layer.shape[1] * scale_factor, input_layer.shape[0] * scale_factor)
            upsampled_layer = cv2.resize(input_layer, new_shape, interpolation=cv2.INTER_LINEAR)
            upsampled_layers.append(upsampled_layer)
        upsampled_img = np.stack(upsampled_layers, axis=1)
        output_data = nib.Nifti1Image(upsampled_img, input_data.affine, input_data.header)
        output_data.header.set_zooms((spacing[0]/scale_factor, spacing[1]/scale_factor, layer_thickness))
        output_name = file_name.replace(".nii.gz", "")
        output_path = os.path.join(output_dir, output_name)
        nib.save(output_data, output_path)